In [ ]:
%pip install openai>/dev/null 
%pip install chromadb>/dev/null 
%pip install langchain>/dev/null

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

In [ ]:
PDF_NAME='Analysis-and-Comparison-between-Optimism-and-StarkNet.pdf'
def load_pdf ():
    return PyMuPDFLoader(PDF_NAME).load()

In [ ]:
docs = load_pdf()

In [ ]:
print(f'You have {len(docs) }document (s) in your data')

print(
    f'There are {len(docs[0].Page_content)}characters in the first page of your document')


total = 0

for doc in docs:

    total + len(doc.Page_content)

print(f'There are {total}characters in your document')
# The total words of the documents.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [ ]:
print(f'Now you have{len(split_docs) } documents')

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
persist_directory = 'starknet'
collection_name = 'starknet'

In [ ]:
from langchain.callbacks import get_openai_callback

In [ ]:
with get_openai_callback() as cb:
    vectorstore = Chroma.from_documents(
        split_docs, embeddings, collection_name=collection_name, persist_directory=persist_directory)
    vectorstore.persist()
    print(cb)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")


# Load the vectorstore from disk
vectordb = Chroma(collection_name=collection_name, persist_directory=persist_directory,
                  embedding_function=embeddings)


query = "What is starknet? "
docs = vectorstore.similarity_search(query, 3, include_metadata=True)

In [ ]:
# 打印chain的提示词
print(chain.document_prompt)

In [ ]:
for doc in docs:
    print(doc.metadata)

In [ ]:
with get_openai_callback() as cb:
    print(chain.run(input_documents=docs, question=query))
    print(cb)